RCSB PDB에서 다운받은 cif 파일을 pdb 파일로 변환

In [4]:
from pathlib import Path
import gemmi

# 변환할 최상위 디렉터리 (여러 단백질 폴더가 들어있는 상위 폴더)
base_dir = Path("/home/yongsu/eupatilin/candidate_1-2/JNK2/7N8T")

# .cif → .pdb 일괄 변환
for cif_file in base_dir.rglob("*.cif"):  # 하위 폴더 포함 모든 .cif 탐색
    try:
        structure = gemmi.read_structure(str(cif_file))
        pdb_file = cif_file.with_suffix(".pdb")
        structure.write_pdb(str(pdb_file))
        print(f"[OK] 변환 완료: {pdb_file}")
    except Exception as e:
        print(f"[ERROR] {cif_file}: {e}")

print("\n✅ 모든 CIF → PDB 변환 완료!")


[OK] 변환 완료: /home/yongsu/eupatilin/candidate_1-2/JNK2/7N8T/7N8T.pdb

✅ 모든 CIF → PDB 변환 완료!


단백질 / 리간드 분리

In [ ]:
import gemmi
from pathlib import Path

# ===== 기본 설정 =====
base_dir = Path("/home/yongsu/eupatilin/candidate_1-3/AKT1/3MV5")

standard_aas = {
    "ALA", "ARG", "ASN", "ASP", "CYS",
    "GLU", "GLN", "GLY", "HIS", "ILE",
    "LEU", "LYS", "MET", "PHE", "PRO",
    "SER", "THR", "TRP", "TYR", "VAL"
}

modified_residues = {
    "MSE", "TPO", "SEP", "PTR", "YCM", "CSO", "CME", "MLY",
    "CSX", "CSD", "HIP", "HID", "HIE", "HYP"
}

# 완전히 제거할 것들 (어느 파일에도 안 들어감)

# 물/용매
solvent_like = {"HOH", "WAT", "DOD", "SOL", "TIP3"}

# 이온들
ion_like = {
    "NA", "K", "CL", "BR", "I",
    "CA", "MG", "MN", "FE", "FE2", "ZN", "CU", "CO",
    "NI", "CD", "HG", "SR", "CS", "LI", "BA", "YB"
}

# 🔹 버퍼/첨가제(통째로 제거하고 싶은 것들)
buffer_like = {
    # 버퍼 계열
    "HEP", "EPE",  # HEPES
    "TRS",         # TRIS
    "MES",         # MES
    "PIP",         # PIPES
    "MPO",         # MOPS
    "ADA",         # ADA 등 필요시 추가

    # 소형 유기 분자 (보통 cryo/결정화 첨가제)
    "GOL",         # glycerol
    "PEG", "PG4", "PG5", "PG6",  # PEG 계열
    "ACT", "ACE",  # acetate
    "CIT",         # citrate
    "FMT",         # formate
    "SO4", "PO4",  # sulfate/phosphate (완전히 버리고 싶으면)
    "EOH",         # ethanol
    "IPA",         # isopropanol
    "DMS"          # DMSO
}

# 여기서 전부 합쳐서 "완전히 제거" 그룹으로 사용
exclude_residues = solvent_like | ion_like | buffer_like


# ===== PDB 파일 순회 =====
for pdb_file in base_dir.rglob("*.pdb"):
    if pdb_file.name.endswith("_protein.pdb") or pdb_file.name.endswith("_ligand.pdb"):
        continue

    try:
        print(f"[INFO] 처리 중: {pdb_file}")

        # Gemmi 구조 읽기
        st = gemmi.read_structure(str(pdb_file))

        prot_st = gemmi.Structure()
        lig_st = gemmi.Structure()
        prot_st.name = st.name
        lig_st.name = st.name

        parsed_residues = set()  # Gemmi에서 처리된 residue 기록

        # --- ① Gemmi 계층 기반 탐색 ---
        for model in st:
            prot_model = gemmi.Model(model.name)
            lig_model = gemmi.Model(model.name)
            for chain in model:
                chain_name = chain.name.strip() if chain.name.strip() else "_"
                prot_chain = gemmi.Chain(chain_name)
                lig_chain = gemmi.Chain(chain_name)
                for res in chain:
                    resname = res.name.strip().upper()
                    parsed_residues.add(resname)
                    if resname in standard_aas or resname in modified_residues:
                        prot_chain.add_residue(res)
                    elif resname not in exclude_residues:
                        lig_chain.add_residue(res)
                if len(prot_chain):
                    prot_model.add_chain(prot_chain)
                if len(lig_chain):
                    lig_model.add_chain(lig_chain)
            if len(prot_model):
                prot_st.add_model(prot_model)
            if len(lig_model):
                lig_st.add_model(lig_model)

        # --- ② line-based 보충 탐색 (계층에 없는 residue 보완) ---
        unparsed_lines = []
        with open(pdb_file) as f:
            for line in f:
                if not line.startswith(("ATOM", "HETATM")):
                    continue
                resname = line[17:20].strip().upper()
                if resname not in parsed_residues:
                    unparsed_lines.append(line)

        if unparsed_lines:
            print(f"[WARN] 계층 구조에서 누락된 원자 {len(unparsed_lines)}개 → line 기반으로 보완")
            for line in unparsed_lines:
                resname = line[17:20].strip().upper()
                if resname in standard_aas or resname in modified_residues:
                    # 단백질 보충
                    model = prot_st[0] if len(prot_st) > 0 else prot_st.add_model("0")
                    chain = model.find_chain("_") or model.add_chain("_")
                    residue = gemmi.make_residue("UNK", 0, ' ')
                    residue.name = resname
                    atom = gemmi.make_small_atom(line[12:16].strip(), float(line[30:38]), float(line[38:46]), float(line[46:54]))
                    residue.add_atom(atom)
                    chain.add_residue(residue)
                elif resname not in exclude_residues:
                    # 리간드 보충
                    model = lig_st[0] if len(lig_st) > 0 else lig_st.add_model("0")
                    chain = model.find_chain("_") or model.add_chain("_")
                    residue = gemmi.make_residue("UNK", 0, ' ')
                    residue.name = resname
                    atom = gemmi.make_small_atom(line[12:16].strip(), float(line[30:38]), float(line[38:46]), float(line[46:54]))
                    residue.add_atom(atom)
                    chain.add_residue(residue)

        # --- ③ 결과 저장 ---
        protein_out = pdb_file.parent / f"{pdb_file.stem}_protein.pdb"
        ligand_out  = pdb_file.parent / f"{pdb_file.stem}_ligand.pdb"

        if len(prot_st) > 0:
            prot_st.write_minimal_pdb(str(protein_out))
            print(f"✅ 단백질 저장 완료: {protein_out}")
        if len(lig_st) > 0:
            lig_st.write_minimal_pdb(str(ligand_out))
            print(f"✅ 리간드 저장 완료: {ligand_out}")

        print()

    except Exception as e:
        print(f"❌ 오류 발생 ({pdb_file.name}): {e}")




[INFO] 처리 중: /home/yongsu/eupatilin/candidate_1-3/AKT1/3MV5/3MV5.pdb
✅ 단백질 저장 완료: /home/yongsu/eupatilin/candidate_1-3/AKT1/3MV5/3MV5_protein.pdb
✅ 리간드 저장 완료: /home/yongsu/eupatilin/candidate_1-3/AKT1/3MV5/3MV5_ligand.pdb



homodimer 단백질 chain 분리

In [1]:
import gemmi
from pathlib import Path

# ====== 입력 파일 ======
input_pdb = Path("/home/yongsu/eupatilin/candidate_1-2/JNK2/3NPC/3NPC_protein.pdb")
output_pdb = input_pdb.with_name(f"{input_pdb.stem}_chain_A.pdb")

# ====== PDB 로드 ======
st = gemmi.read_structure(str(input_pdb))
model = st[0]
chains = [c for c in model]
print(f"[INFO] 체인 수: {len(chains)}")

def get_ca_coords(chain):
    """Cα 좌표만 추출"""
    return [atom.pos for res in chain for atom in res if atom.name.strip() == "CA"]

def compute_rmsd(coords1, coords2):
    """좌표 수 달라도 최소 공통 길이만큼 RMSD 계산"""
    n = min(len(coords1), len(coords2))
    if n == 0:
        return None
    pos1 = coords1[:n]
    pos2 = coords2[:n]
    sup = gemmi.superpose_positions(pos1, pos2)
    return sup.rmsd

# ====== 기준 체인 (A) ======
ref_chain = chains[0]
ref_coords = get_ca_coords(ref_chain)
if not ref_coords:
    raise ValueError("기준 체인에 Cα 좌표가 없습니다.")

# ====== 모든 체인 RMSD 계산 (A 기준) ======
for chain in chains[1:]:
    test_coords = get_ca_coords(chain)
    if not test_coords:
        print(f"[WARN] {chain.name}: Cα 좌표 없음 → 건너뜀")
        continue
    rmsd = compute_rmsd(ref_coords, test_coords)
    if rmsd is None:
        print(f"[WARN] {chain.name}: 비교 불가 (좌표 없음)")
        continue
    print(f"[INFO] Chain {chain.name} RMSD vs A = {rmsd:.3f} Å")

# ====== 결과 저장: A 체인만 남김 ======
new_st = gemmi.Structure()
new_st.name = st.name
new_model = gemmi.Model(model.name)
new_model.add_chain(ref_chain)
new_st.add_model(new_model)
new_st.write_minimal_pdb(str(output_pdb))

print(f"\n[OK] Chain A만 남긴 구조 저장 완료 → {output_pdb}")



[INFO] 체인 수: 2
[INFO] Chain B RMSD vs A = 3.813 Å

[OK] Chain A만 남긴 구조 저장 완료 → /home/yongsu/eupatilin/candidate_1-2/JNK2/3NPC/3NPC_protein_chain_A.pdb


단백질 pdb 파일 pdbqt로 변환

In [ ]:
prepare_receptor4.py -r protein.pdb -o protein.pdbqt -A checkhydrogens

In [3]:
'''alternate location 원자가 존재할 경우 아래 코드 실행하여 점유율 높은 alt 만 남긴 후에 다시 변환'''

from pathlib import Path

input_pdb = Path("/home/yongsu/eupatilin/candidate_1-2/JNK2/7N8T/7N8T_protein.pdb")
output_pdb = input_pdb.with_name(f"{input_pdb.stem}_altclean.pdb")

# occupancy 비교용 딕셔너리
atoms_by_key = {}
occupancies = {}

with open(input_pdb) as f:
    lines = f.readlines()

for line in lines:
    if not line.startswith(("ATOM", "HETATM")):
        continue

    altloc = line[16].strip()  # column 17 (A/B/C 등)
    resname = line[17:20].strip()
    chain = line[21].strip()
    resnum = line[22:26].strip()
    atom_name = line[12:16].strip()
    occ_str = line[54:60].strip()
    occ = float(occ_str) if occ_str else 0.0

    key = (chain, resnum, resname, atom_name)

    # ALTLOC이 없는 경우는 occupancy = 1.0으로 간주해 그대로 포함
    if not altloc:
        occ = 1.0

    # 점유율이 더 높으면 교체
    if key not in occupancies or occ > occupancies[key]:
        atoms_by_key[key] = line
        occupancies[key] = occ

# 새 파일 작성
with open(output_pdb, "w") as out:
    for line in lines:
        # HEADER, REMARK 등 비원자 라인은 그대로 유지
        if not line.startswith(("ATOM", "HETATM")):
            out.write(line)
    for atom_line in atoms_by_key.values():
        # ALTLOC 문자는 공백으로 바꿔 표준 PDB 형식 유지
        atom_line = atom_line[:16] + " " + atom_line[17:]
        out.write(atom_line)

print(f"[OK] 점유율(occupancy) 높은 conformation만 남긴 파일 저장 완료: {output_pdb}")



[OK] 점유율(occupancy) 높은 conformation만 남긴 파일 저장 완료: /home/yongsu/eupatilin/candidate_1-2/JNK2/7N8T/7N8T_protein_altclean.pdb


Docking

In [7]:
from pathlib import Path
import subprocess, sys, re
from rdkit import Chem
from meeko import MoleculePreparation, PDBQTWriterLegacy
import pandas as pd


# ====== 경로 설정 ======
PROTEIN_PDBQT = Path("/home/yongsu/eupatilin/candidate_1-2/JNK2/7N8T/7N8T_protein.pdbqt")
LIGAND_PDB    = Path("/home/yongsu/eupatilin/candidate_1-2/JNK2/7N8T/7N8T_ligand.pdb")
COMPOUND      = Path("/home/yongsu/eupatilin/candidate_1-3/eupatilin.sdf")
SMINA_BIN     = "/home/yongsu/eupatilin/candidate_1-3/smina.linux"
OUT_DIR       = Path("/home/yongsu/eupatilin/candidate_1-2/JNK2/7N8T")
OUT_DIR.mkdir(exist_ok=True, parents=True)

# ====== 파일명 설정 ======
protein_name  = OUT_DIR.name
compound_name = COMPOUND.stem
base_stem     = f"{compound_name}_{protein_name}"
print(f"[INFO] 출력 파일 prefix: {base_stem}")


# ====== 포켓 중심 & 크기 계산 ======
def calc_box_from_ligand(lig_path: Path, margin: float = 2.0):
    xs, ys, zs = [], [], []

    with open(lig_path) as f:
        for line in f:
            if line.startswith(("ATOM", "HETATM")):
                xs.append(float(line[30:38].strip()))
                ys.append(float(line[38:46].strip()))
                zs.append(float(line[46:54].strip()))

    if not xs:
        raise ValueError(f"{lig_path.name}에서 원자 좌표를 읽을 수 없음")

    minx, maxx = min(xs), max(xs)
    miny, maxy = min(ys), max(ys)
    minz, maxz = min(zs), max(zs)

    center = [(minx+maxx)/2, (miny+maxy)/2, (minz+maxz)/2]
    size = [(maxx-minx)+margin, (maxy-miny)+margin, (maxz-minz)+margin]

    print(f"[INFO] center = {center}")
    print(f"[INFO] size   = {size}")
    return center, size


center, size = calc_box_from_ligand(LIGAND_PDB, margin=2.0)


# ====== SDF → PDBQT 변환 ======
def sdf_to_pdbqt(sdf_path: Path) -> Path:
    suppl = Chem.SDMolSupplier(str(sdf_path), removeHs=False)
    mol = next((m for m in suppl if m is not None), None)
    if mol is None:
        raise ValueError(f"{sdf_path}에서 분자를 읽을 수 없음")

    prep = MoleculePreparation()
    setup = list(prep.prepare(mol))[0]
    writer = PDBQTWriterLegacy()
    out_str = writer.write_string(setup)
    if isinstance(out_str, tuple):
        out_str = out_str[0]

    out_path = sdf_path.with_suffix(".pdbqt")
    out_path.write_text(out_str)
    print(f"[INFO] 리간드 변환 완료 → {out_path}")
    return out_path


COMPOUND_PDBQT = sdf_to_pdbqt(COMPOUND)


# ====== 도킹 실행 ======
def run_smina(ligand, receptor, center, size, out_dir):
    out_pose = out_dir / f"{base_stem}_docked.pdbqt"
    log_file = out_dir / f"{base_stem}_dock.log"

    cmd = [
        SMINA_BIN,
        "--ligand", str(ligand),
        "--receptor", str(receptor),
        "--center_x", str(center[0]),
        "--center_y", str(center[1]),
        "--center_z", str(center[2]),
        "--size_x", str(size[0]),
        "--size_y", str(size[1]),
        "--size_z", str(size[2]),
        "--exhaustiveness", "8",
        "--num_modes", "10",
        "--seed", "42",
        "--out", str(out_pose),
        "--log", str(log_file)
    ]

    print(f"[RUN] 도킹 시작...")
    with open(log_file, "w") as lf:
        proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
        for line in proc.stdout:
            sys.stdout.write(line)
            lf.write(line)
        proc.wait()

    print(f"[OK] 도킹 완료 → {out_pose}")
    return out_pose, log_file


pose_file, log_file = run_smina(COMPOUND_PDBQT, PROTEIN_PDBQT, center, size, OUT_DIR)


# ====== 로그에서 affinity 파싱 ======
def parse_affinity(log_path):
    rows = []
    txt = Path(log_path).read_text()

    pattern = r"^\s*(\d+)\s+([\-\d\.]+)\s+([\-\d\.]+)\s+([\-\d\.]+)"
    for m in re.finditer(pattern, txt, re.MULTILINE):
        pose = int(m.group(1))
        affinity = float(m.group(2))
        rmsd_lb = float(m.group(3))
        rmsd_ub = float(m.group(4))
        rows.append([pose, affinity, rmsd_lb, rmsd_ub])

    return rows


rows = parse_affinity(log_file)

df = pd.DataFrame(rows, columns=["pose", "affinity_kcal/mol", "RMSD_lb", "RMSD_ub"])
df = df.sort_values("affinity_kcal/mol", ascending=True)
csv_path = OUT_DIR / f"{base_stem}_docked_summary.csv"
df.to_csv(csv_path, index=False)

print(f"\n[OK] CSV 저장 완료 → {csv_path}")


[INFO] 출력 파일 prefix: eupatilin_7N8T
[INFO] center = [5.6535, 0.41949999999999976, -9.0825]
[INFO] size   = [5.6370000000000005, 10.940999999999999, 10.337]
[INFO] 리간드 변환 완료 → /home/yongsu/eupatilin/candidate_1-3/eupatilin.pdbqt
[RUN] 도킹 시작...
              _             
             (_)            
   __ _ _ __  _ _ __   __ _ 
  / _` | '_ \| | '_ \ / _` |
 | (_| | | | | | | | | (_| |
  \__, |_| |_|_|_| |_|\__,_|
   __/ |                    
  |___/                     

gnina v1.0.1 HEAD:aa41230   Built Mar 23 2021.
gnina is based on smina and AutoDock Vina.
Please cite appropriately.

Commandline: /home/yongsu/eupatilin/candidate_1-3/smina.linux --ligand /home/yongsu/eupatilin/candidate_1-3/eupatilin.pdbqt --receptor /home/yongsu/eupatilin/candidate_1-2/JNK2/7N8T/7N8T_protein.pdbqt --center_x 5.6535 --center_y 0.41949999999999976 --center_z -9.0825 --size_x 5.6370000000000005 --size_y 10.940999999999999 --size_z 10.337 --exhaustiveness 8 --num_modes 10 --seed 42 --out /home/yongsu/eu

Docking 검증

In [ ]:
# 불러올 파일들 전부 c드라이브의 pymol_script 파일에 있어야 함

# pymol 콘솔창에 입력

# 스크립트 로드
run C:/pymol_scripts/compare_docking_interactions.py

# 구조 로드
load "C:\pymol_scripts\3O96.pdb", exp
load "C:\pymol_scripts\3O96_protein.pdbqt", dockprot
load "C:\pymol_scripts\eupatilin_3O96_docked.pdbqt", docklig

# 비교 실행 (cutoff 4.0 Å 예시)
compare_docking_interactions exp, dockprot, docklig, 4.0